# How to compare several fields in one plot

The SciDataTool python module has been created to **ease the handling of scientific data**, and considerately simplify plot commands. It unifies the extraction of relevant data (e.g. slices), whether they are stored in the time/space or in the frequency domain. The call to Fourier Transform functions is **transparent**, although it still can be parameterized through the use of a dictionary.

This tutorial explains the how to **compare** several fields, even if they are not defined with the same bases, or even if they were stored in different time/frequency domains.

After learning [how to create `Data` objects](https://nbviewer.jupyter.org/github/Eomys/SciDataTool/blob/master/Tutorials/tuto1_Create.ipynb) and [how to extract slices](https://nbviewer.jupyter.org/github/Eomys/SciDataTool/blob/master/Tutorials/tuto2_Slices.ipynb), you may want to **compare on a same graph** several fields. The `compare_along` methods will provide you a very practical tool to do so, as you can see in this first example:

In [7]:
# import SciDataTool objects
from SciDataTool import Data1D, DataLinspace, DataTime, DataFreq, VectorField

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

Time = DataLinspace(
    name="time",
    unit="s",
    initial=0,
    final=0.1,
    number=100,
    include_endpoint=False,
)
field = np.cos(2 * np.pi * 10 * Time.get_values())
Field = DataTime(
    name="Example field",
    symbol="X",
    axes=[Time],
    values=field,
    unit="m/s2",
)
Time2 = DataLinspace(
    name="time",
    unit="s",
    initial=0.01,
    final=0.09,
    number=10,
    include_endpoint=False,
)
field2 = np.sin(2 * np.pi * 10 * Time2.get_values())
Field2 = DataTime(
    name="Other example field",
    symbol="Y",
    axes=[Time2],
    values=field2,
    unit="m/s2",
)

#---------------------------------------------------------------
# Compare the fields
result = Field.compare_along("time", data_list=[Field2])
#---------------------------------------------------------------


# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=result["time"],
    y=result[Field.symbol],
    name=r"$"+Field.symbol+"$"
))
fig.add_trace(go.Scatter(
    x=result["time"],
    y=result[Field2.symbol + "_0"],
    name=r"$"+Field2.symbol+"$"
))
fig.update_layout(
    title={
        'text': Field.name + " comparison",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Time [s]",
    yaxis_title=r"$"+Field.symbol+" ["+Field.unit+"]$",
)
fig.show(config = {"displaylogo":False})

The `Data` classes allow to easily compare several fields in the same graph, even if they don't have the same basis or are stored in different time/frequency domains. To do so, the `compare_along` methods find a common basis for the axes requested, and interpolate the fields on this common basis. The syntax for requesting axes, unit and normalization, is the same as in a slice with the `get_along` methods.

An additional argument appears: the `data_list` is a **list** that contains the `Data` objects to compare to the original field.

Note that if `data_list` is not specified, the `get_along` method will simply be called, so that this method can be used **by default** (which can be useful to define **generic** plot methods). Note also that `compare_FT_along`, `compare_magnitude_along` and `compare_phase_along` are available for **comparisons of spectra**.